In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API keys
from api_key import weather_api_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Store file path
csv_path="Output/Border_Crossing_Final_Data.csv"

# Read the csv file
weather_df = pd.read_csv(csv_path).drop(columns="Index")

# Display the dataset
weather_df.head()

,Year,Month,Timestamp,Border,Port Name,State,Measure,Value,Longitude,Latitude
0,2018,12,1543622400,US-Canada Border,Van Buren,Maine,Trucks,1204,-67.94271,47.16207
1,2018,12,1543622400,US-Canada Border,Maida,North Dakota,Trucks,170,-98.36953,48.98568
2,2018,12,1543622400,US-Mexico Border,Douglas,Arizona,Buses,210,-109.54472,31.34444
3,2018,12,1543622400,US-Mexico Border,Presidio,Texas,Bus Passengers,238,-104.37167,29.56056
4,2018,12,1543622400,US-Canada Border,Anacortes,Washington,Personal Vehicle Passengers,5350,-122.61739,48.49988


### Retrieve Weather Data for 2018 from OpenWeatherMap API

In [3]:
# Filter the dataset for only last year needed for analysis (2018)
weather_df = weather_df.loc[(weather_df["Year"] == 2018)]

# Display the filtered dataset
weather_df.head()

,Year,Month,Timestamp,Border,Port Name,State,Measure,Value,Longitude,Latitude
0,2018,12,1543622400,US-Canada Border,Van Buren,Maine,Trucks,1204,-67.94271,47.16207
1,2018,12,1543622400,US-Canada Border,Maida,North Dakota,Trucks,170,-98.36953,48.98568
2,2018,12,1543622400,US-Mexico Border,Douglas,Arizona,Buses,210,-109.54472,31.34444
3,2018,12,1543622400,US-Mexico Border,Presidio,Texas,Bus Passengers,238,-104.37167,29.56056
4,2018,12,1543622400,US-Canada Border,Anacortes,Washington,Personal Vehicle Passengers,5350,-122.61739,48.49988


In [4]:
# Groupby 'Port Name' to create a new dataframe to record the Nearest City for each Port
port_data = weather_df.groupby(["Port Name", "Timestamp", "Year", "Month", "Border", "Latitude", "Longitude"])["Value"].sum().reset_index()

# Display the dataset
port_data.head()

,Port Name,Timestamp,Year,Month,Border,Latitude,Longitude,Value
0,Alcan,1514764800,2018,1,US-Canada Border,63.33693,-142.98928,3809
1,Alcan,1517443200,2018,2,US-Canada Border,63.33693,-142.98928,3759
2,Alcan,1519862400,2018,3,US-Canada Border,63.33693,-142.98928,5609
3,Alcan,1522540800,2018,4,US-Canada Border,63.33693,-142.98928,8924
4,Alcan,1525132800,2018,5,US-Canada Border,63.33693,-142.98928,21087


In [5]:
# Add empty columns to the DataFrame to store weather data from OpenWeatherMap API
port_data[["Temperature", "Cloudiness", "Humidity", "Wind Speed", "Weather condition"]] = ""

# Display the sample dataset:
port_data.head()

,Port Name,Timestamp,Year,Month,Border,Latitude,Longitude,Value,Temperature,Cloudiness,Humidity,Wind Speed,Weather condition
0,Alcan,1514764800,2018,1,US-Canada Border,63.33693,-142.98928,3809,,,,,
1,Alcan,1517443200,2018,2,US-Canada Border,63.33693,-142.98928,3759,,,,,
2,Alcan,1519862400,2018,3,US-Canada Border,63.33693,-142.98928,5609,,,,,
3,Alcan,1522540800,2018,4,US-Canada Border,63.33693,-142.98928,8924,,,,,
4,Alcan,1525132800,2018,5,US-Canada Border,63.33693,-142.98928,21087,,,,,


In [15]:
# Set the API base URL
url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?"

# Define units to get the temperature in Celsius
units = "metric"

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through the dataset to fetch weather data
for index, row in port_data.iterrows():

    # Get latitude, longitude from the DataFrame
    lat = port_data.loc[index, "Latitude"]
    lon = port_data.loc[index, "Longitude"]
    time = port_data.loc[index, "Timestamp"]

    # Create endpoint URL with each port's latitude and logitude values
    port_url = f"{url}appid={weather_api_key}&lat={lat}&lon={lon}&units={units}&dt={time}"

    # Run an API request for each of the ports
    try:
        # Parse the JSON and retrieve data
        port_weather = requests.get(port_url).json()

            # Note: We used pprint to review the response
            # pprint(port_weather)

        # Parse out temperature, cloudiness, humidity, wind speed and weather condition
        port_data.loc[index, "Temperature"] = port_weather["data"][0]["temp"]
        port_data.loc[index, "Cloudiness"] = port_weather["data"][0]["clouds"]
        port_data.loc[index, "Humidity"] = port_weather["data"][0]["humidity"]
        port_data.loc[index, "Wind Speed"] = port_weather["data"][0]["wind_speed"]
        port_data.loc[index, "Weather condition"] = port_weather["data"][0]["weather"][0]["main"]

    # If an error is experienced, skip the port
    except:
        print("Port not found. Skipping...")
        pass

# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
-----------------------------
Data Retrieval Complete      
-----------------------------


In [16]:
# Show the data
port_data.head()

,Port Name,Timestamp,Year,Month,Border,Latitude,Longitude,Value,Temperature,Cloudiness,Humidity,Wind Speed,Weather condition,Himidity
0,Alcan,1514764800,2018,1,US-Canada Border,63.33693,-142.98928,3809,-25.35,100,73,1.59,Clouds,73.0
1,Alcan,1517443200,2018,2,US-Canada Border,63.33693,-142.98928,3759,-28.87,17,84,1.02,Clouds,84.0
2,Alcan,1519862400,2018,3,US-Canada Border,63.33693,-142.98928,5609,-13.19,0,59,3.47,Clear,59.0
3,Alcan,1522540800,2018,4,US-Canada Border,63.33693,-142.98928,8924,0.09,97,49,2.39,Clouds,49.0
4,Alcan,1525132800,2018,5,US-Canada Border,63.33693,-142.98928,21087,9.32,83,45,5,Clouds,45.0


In [17]:
# Export the CSV for final weather data
port_data.to_csv('Output/Weather_Data.csv', index_label="Index")